In [1]:
require 'bio'

true

In [91]:
# Convert sequences to protein file
faaoutput = File.open('./data/athaliana.faa', 'w')
fasta_file = Bio::FlatFile.auto("./data/athaliana.fa")
fasta_file.each_entry do |x|
    if x.naseq.translate[-1] == "*"
        protein = x.naseq.translate[1..-2]
    else
        protein = x.naseq.translate
    end
    seq_str = protein.to_s
    seq = Bio::Sequence.new(seq_str)
    faaoutput.puts seq.output_fasta(definition = x.definition)
end

In [89]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end

In [ ]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|å
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end

In [1]:
# Retrieve query sequences
require 'rest-client'
require 'bio'

# Auxiliary function to fetch a URL correctly.
# 
# @param url [String] The URL to search.
# @return [String] The response for the URL.
def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
	response = RestClient::Request.execute({
	  method: :get,
	  url: url.to_s,
	  user: user,
	  password: pass,
	  headers: headers})
	return response
	
	rescue RestClient::ExceptionWithResponse => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
	rescue RestClient::Exception => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
	rescue Exception => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
end

  
genes = File.open('./ArabidopsisSubNetwork_GeneList.txt', 'r')
genearray = genes.read.split() # this will read each line into an array
genes.close
  
fastaoutput = File.open('./data/ara_genes.fa', 'w')

genearray.each do |geneid|
  
  url = "http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=fasta&id=#{geneid}"
  res = fetch(url)  # we really should check that the return value is valid, but... 
  fastafile = Bio::FlatFile.new(Bio::FastaFormat, StringIO.new(res.body))
  fastafile.each_entry do |fasta|
    naseq = fasta.to_biosequence
    fastaoutput.puts naseq.output_fasta(geneid)
  end
end
puts "DONE"

DONE


In [13]:
# Auxiliary method to index the fasta files
def fasta2dict(fasta_file)
    fasta_dict = Hash.new
    fasta_data = Bio::FlatFile.auto(fasta_file)
    fasta_data.each_entry do |fasta|
        fasta_dict[fasta.definition] = fasta.to_s
    end
    return fasta_dict
end

:fasta2dict

In [14]:
spombe_fasta = fasta2dict("./data/spombe.fa")
puts

In [41]:
require 'bio'

# blast is installed globally, so you don't need to include the full path to blastn
factory = Bio::Blast.local('blastp', './databases/spombe', '-e 0.0001')   # wherever your database is....a BLAST "factory" 
#factory = Bio::Blast.local('tblastx', '/home/osboxes/Databases/EBOLA') # 
# Matches using proteins are more biologically meaningfull!! More AA (DNA is more redundant)
# blastn is the least accurate.

query_file = "./data/athaliana.faa"
query_data = Bio::FlatFile.auto(query_file)

reciprocal_factory = Bio::Blast.local('blastp', './databases/athaliana_prot', '-e 0.000001')
#reciprocal_query_file = "./data/spombe.faa"
#reciprocal_query_data = Bio::FlatFile.auto(query_file)

id_regex = Regexp.new(/^(\w+\.\w+)/i)

File.open('RBH_ara_spombe.txt','w') do |file|
  file.write("Orthologues between A.thaniana and S.pombe\n")
  file.write("A.thaniana gene\tS.pombe gene\n")

  query_data.each_entry do |query_fasta|

    # puts query_fasta.entry_id
    query = query_fasta.to_s
    # puts query.length
    report = factory.query(query) # Bio::Blast::Report
    hit = report.hits[0] # hit: Bio::Blast::Report::Hit
    next if hit.nil? # Filter no hits
    # hit: Bio::Blast::Report::Hit
    # puts "hit: query #{hit.query_def} with target #{hit.target_def}"
    # puts "Eval #{hit.evalue}\tScore #{hit.bit_score/query.length.to_f}"
    # puts "#{hit.lap_at}"
    hsp = hit.hsps[0] # Bio::Blast::Report::Hsp
    next if hsp.nil?

    cov = hsp.align_len/query_fasta.length.to_f

    if query_fasta.definition.match(id_regex)
      query_id = $1
    end

    if cov > 0.5
      # puts "Query coverage: #{cov}" 
      # puts hsp.qseq
      # puts hsp.midline
      # puts hsp.hseq
      # Make blast with other seq
      reciprocal_factory = Bio::Blast.local('blastp', './databases/athaliana_prot', '-e 0.000001')
      reciprocal_query = Bio::FastaFormat.new(spombe_fasta[hit.target_def])

      reciprocal_query_id = reciprocal_query.entry_id

      reciprocal_report = reciprocal_factory.query(reciprocal_query.to_s) # Bio::Blast::Report

      reciprocal_hit = reciprocal_report.hits[0] # hit: Bio::Blast::Report::Hit
      next if reciprocal_hit.nil? # Filter no hits

      reciprocal_hsp = reciprocal_hit.hsps[0] # Bio::Blast::Report::Hsp
      next if reciprocal_hsp.nil?

      if reciprocal_hit.target_def.match(id_regex)
        reciprocal_target_id = $1
      end

      reciprocal_cov = reciprocal_hsp.align_len/reciprocal_query.length.to_f
      
      if reciprocal_cov > 0.5
        # Check if the target hit is the query hit of the first blast
        if reciprocal_target_id == query_id
          file.write("#{query_id}\t#{reciprocal_query_id}\n")
        end
      end
    end
  end
end
# executes homology search. Returns Bio::Blast::Report object.
# report = factory.query(">myseq\nAGCCCCGTTTTTTGCCACTGGAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT") # this is a Bio::Blast::Report
# This is important for the assignment.

#puts factory.output   # to see the blast report, verbatim
#puts report.hits
# Iterates over each hit
# report.each_hit do |hit|
#   puts hit.class
#   puts "#{hit.hit_id} : evalue #{hit.evalue}\t#{hit.bit_score} at "
#   puts "#{hit.lap_at}"   # this tells you start and end of both the query and the hit sequences
#   hit.each do |hsp|
#     puts hsp.align_len
#     puts hsp.qseq  # this is the gapped Alignment as text of the query
#     puts hsp.hseq  # this is the gapped Alignment as text of the hit
#   end
# end

# Its possible that the query matches different places of the database (it considers it two separate matches).
# high scoring segment pair (both sequence aligned) (hsp).
# In a report you have multiple hits, each hit has multiple hsp

Interrupt: 

In [115]:
spombe_fasta['SPBC651.01c|nog1|II|GTP binding protein Nog1 (predicted)']

">SPBC651.01c|nog1|II|GTP binding protein Nog1 (predicted)\nMATAVFKNITPIPDVNTFLDVVLSRTQRKTPTVIRSGFKISRIRGFYGRKVKFTQDTITE\nKLDSILQEFPKLNDIHPFHADLLNILYDRDHLKIALSQLSTAKHLVENVARDYIRLLKYG\nDSLYRCKQLKRAALGRMATIIKRQKSSLEFLEQVRQHLSRLPAIDPNTRTLLVCGYPNVG\nKSSFMNKVTRAQVDVQPYAFTTKSLFVGHFDYKYLRWQVIDTPGILDHPLEQMNTIEMQS\nITAMAHLRSAVLYFMDLSEMCGYSVAAQVKLYHSIKPLFANKVTILVLNKIDAMRPEDLD\nQKNQELLQTIIDDGNVKVVQTSCVQDIGVMDVRTTACEALLAARVEQKLKGSRVNNVLNR\nIHLAEPAARDEVARPACIPDSVKTRRAYDANDPNRRILARDIEAANGGAGVYNVELRDKY\nILQDPSWKYDRMPELLDGKNVADFVDPEIEAKLLALDEEEERLEREGYYDSDQEIEDADE\nEEVLEKASRIREKNKLTMLAARQKKIKNRPVLPRTAGIRTLDELQSSLQSAGLPSDSIEE\nRARSRARTAAANEMADGSGVDLLMNEGEEIRSKSIAPRSQSNRRESGVHAEGARSYADRL\nARVKQIHRNRMARASESDRHVIAAKPKHLLSGKRGNGKTQRR\n"

In [114]:
require 'stringio'

false

In [8]:
require 'bio'

false